## INSTRUCTIONS

  * Set INTERACTIVE to directory containing 'model_init.py' (probably the same as this script)
  * Set ROOT to the same
  * Load the model and try the examples below

In [1]:
import sys
import random
from collections import defaultdict, namedtuple
from pathlib import Path
from textwrap import indent, dedent

import torch

# location containing 'model_init.py'
#INTERACTIVE = '/data/home/coclemen/projects/method-docstring-webapp/server'
#if INTERACTIVE not in sys.path:
#    sys.path.append(INTERACTIVE)

In [2]:
#from model_init import FairseqModelInterface, Args, generate_method, generate_docstring, code_format_docstring
%run -i /home/coclemen/projects/AthenaDataPipeline/athenadatapipeline/model_init.py

In [5]:
ROOT = Path("/smartml-athena/processed-data/fundef-docstrings/split/multi-task-translation/athena-common/")
TOKENIZER_PREFIX = str(ROOT / "hacky_byte_level")

args = Args(
        model_name_or_path=str(ROOT),
        data_name_or_path=str(ROOT),
        tokenizer_prefix=TOKENIZER_PREFIX,
        checkpoint_file='checkpoint_best.pt',
)

model = FairseqModelInterface()
model.load_model(args)

loading archive file /smartml-athena/processed-data/fundef-docstrings/split/multi-task-translation/athena-common
loading archive file /smartml-athena/processed-data/fundef-docstrings/split/multi-task-translation/athena-common
| [src] dictionary: 50181 types
| [tgt] dictionary: 50181 types
Namespace(activation_dropout=0.2, activation_fn='relu', adam_betas='(0.9,0.98)', adam_eps=1e-06, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.1, best_checkpoint_metric='loss', bpe=None, broadcast_buffers=False, bucket_cap_mb=25, clip_norm=0.1, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='/smartml-athena/processed-data/fundef-docstrings/split/multi-task-translation/athena-common', dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=1472, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1472, d

In [ ]:
from fairseq.search import Sampling, DiverseBeamSearch, DiverseSiblingsSearch

In [6]:
example = [
    'def merge_sorted_lists(left, right):',  # signature
    'Return one merged sorted list',  # docstring
    'result = []',
    #'merged = []\nif len(first) < len(second):\n    (first, second) = (second, first)\nfor i'  # partial body
]

In [7]:
method = generate_method(example, model, sampling=True, sampling_topk=20, sampling_topp=.75)
print(example[0])  # signature
print(code_format_docstring(example[1]))
print(method)

def merge_sorted_lists(left, right):
    """Return one merged sorted list"""
    left = sorted(left)
    right = sorted(right)
    i = 0
    while (i < len(left)):
        if (left[i] < right[i]):
            (left[i], right[i]) = (right[i], left[i])
            i += 1
        else:
            i += 1
    return left


In [28]:
example = [
    '',
    'flatten a list of lists into one list',
    ''
]
method = generate_method(example, model, beam=6)
print(method)

def flatten(lst):
    return [item for sublist in lst for item in sublist]


In [29]:
def flatten(lst):
    return [item for sublist in lst for item in sublist]

In [ ]:
example[2] = method
print(generate_docstring(example, model, style='numpydoc'))

## Some OPENAI Codegen examples

In [ ]:
OPENAI_CODEGEN = {}
OPENAI_CODEGEN['locate-3s'] = [
        'def return_indices_of_list_of_elements_that_are_3(l):',
        'Given the list l, return the indices of those list\n elements that have the value 3.',
        ''
]


OPENAI_CODEGEN['print-first-3'] = [
        'def print_first_3_characters_of_string(s):',
        '',
        ''
]

OPENAI_CODEGEN['first-3-to-int'] = [
        'def convert_first_3_characters_of_string_s_to_number(s):',
        '',
        ''
]

OPENAI_CODEGEN['reverse'] = [
        'def reverse_order_of_words(s):',
        '',
        ''
]

OPENAI_CODEGEN['reverse-for-loop'] = [
        'def reverse_list_using_explicit_for_loop(l):',
        '',
        ''
    
]

OPENAI_CODEGEN['return-odd'] = [
        'def return_odd_numbers_from_list(l):',
        'Given a list of integers, return only\n those which are odd.',
        ''
]


OPENAI_CODEGEN['return-odd-index-sum'] = [
        'def return_sum_of_odd_numbered_elements_of_list(l):',
        'Given the list l, return the sum of the elements\nthat occur at an odd-numbered index.',
        ''
]

OPENAI_CODEGEN['format-grades'] = [
        'def format_grades(f):',
        (
            'The file f contains the midterm grades for all the student in the class.\n'
            'Each line in the file has the format:\n'
            '    <Last Name>, "\t", <First Name>, "\t", <Score>\n'
            'This function reads the file and prints out each line in the format:\n'
            '    <Last Name>, "\t", <First Name>, "\t", <Score>\n'
        ),
        ''
]

OPENAI_CODEGEN['ask-user'] = [
        'def main():',
        'Asks the user for a number n and then\nprints the sum of all the numbers from 1 to n.',
        ''    
]

OPENAI_CODEGEN['alice-or-bob'] = [
        'def main():',
        'Asks the user for their name and then greets\nthem with their name, if their name is Alice or Bob.',
        ''
]

In [ ]:
examples = OPENAI_CODEGEN
print(examples.keys())
for name, example in examples.items():
    sig, docstring, _ = example
    method = generate_method(example, model, beam=5, temperature=1.0)
    print('\n'.join([sig, code_format_docstring(docstring), method]))
    print('\n')

## More OpenAI codegen examples

In [ ]:
%run -i "aicodegen_examples.py"

In [ ]:
examples = OPENAI_CODEGEN_EXAMPLES
print(examples.keys())
for name, example in examples.items():
    sig, docstring, _ = example
    method = generate_method(example, model, beam=5, temperature=1.0)
    print('\n'.join([sig, code_format_docstring(docstring), method]))
    print('\n')

## More examples for playing

In [33]:
examples = [
    'def print_unique_lines(file_path):',  # signature
    'print the lines of file_path which occur only once',  # docstring
    ''  # body
]
%time print(generate_method(examples, model))

    lines = []
    with open(file_path, 'r') as f:
        for line in f:
            if (line not in lines):
                lines.append(line)
    return lines
CPU times: user 618 ms, sys: 31.9 ms, total: 649 ms
Wall time: 648 ms


In [38]:
examples = [
    'def fibonacci(n):',  # signature
    '', #'sum elements of list which are odd'  # docstring
    ''
]
print(generate_method(examples, model))

    if (n == 1):
        return 1
    elif (n == 2):
        return 2
    else:
        return (fibonacci((n - 1)) + fibonacci((n - 2)))


In [ ]:
example = [
    'def sum_odd_elements(lst):',  # signature
    'sum elements of list which are odd',  # docstring
    '''sum = 0
for i in range(len(lst)):
    if (lst[i] % 2):
        sum += lst[i]
return sum
'''
]
print(generate_docstring(example, style='javadoc'))